# Mode Selection
Set finetuning = 1 if you want to finetune, or set finetuning = 0 if you want to just load the saved model to get inference.

In [1]:
finetuning = 0
load_and_run = not finetuning

# Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

In [3]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/#Research/# GB/train_test_hatespeech_data.csv')
data.dropna(inplace = True)
data

,male,female,_original_label,test
0,যত্তসব পাপন শালার ফাজলামী!!!!!,যত্তসব জাহান শালীর ফাজলামী!!!!!,1,0
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,নাহার শালী রে রিমান্ডে নেওয়া দরকার,1,0
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,নূর জাহান ম্যাডামের মেয়ে এতো বড় জারজ হবে এটা ...,1,0
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,শালী লুচ্চা দেখতে পাঠীর মত দেখা যায়,1,1
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,তুই তো শালী গাজা খাইছচ । তুর মার হেডায় খেলবে স...,1,0
...,...,...,...,...
9997,ভাই এরকম ভিডিও আরো করেন,আফা এরকম ভিডিও আরো করেন,0,0
9998,ভাই এরকম ভিডিও আরো করেন,বোন এরকম ভিডিও আরো করেন,0,0
9999,ভাই এরকম ভিডিও আরো করেন,ভাবী এরকম ভিডিও আরো করেন,0,0
10000,মূর্খ খালেদার ফাঁসি চাই,মূর্খা দুর্গা ফাঁসি চাই,0,0


In [4]:
test = data[data.test==1]
test

,male,female,_original_label,test
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,শালী লুচ্চা দেখতে পাঠীর মত দেখা যায়,1,1
8,দেখেছো মাইগ্যা না হিজরা বোঝা যাচ্ছে না । দেখে...,দেখেছো মাগি না হিজরা বোঝা যাচ্ছে না। দেখে মনে ...,1,1
10,সব মানুষ আপন হয় না জেমন পাপন আপন হইতে পারে না ...,সব মানুষ আপন হয় না জেমন রাইসা আপন হইতে পারে না...,1,1
17,এই চুদির ভাই তুই কোথায় শুনলি সাকিব ইংল্যান্ডেে...,এই চুদির আফা তুই কোথায় শুনলি সাকিব ইংল্যান্ডেে...,1,1
20,পাপন সালা পাগল হয়ে গেছে সবাই মিলে একে জুতা মারো,সাহানা সালী পাগলী হয়ে গেছে সবাই মিলে একে জুতা...,1,1
...,...,...,...,...
9961,স্বামী স্ত্রী কে ভালবেসে মিষ্টি খাওয়াইছে বলেএই...,স্ত্রী স্বামী কে ভালবেসে মিষ্টি খাওয়াইছে বলেএই...,0,1
9966,রানু পাগলি কে খমা করা উচিত নয়,আহনাফ পাগলা কে খমা করা উচিত নয়,0,1
9969,তাহসানের উচিৎ ভালো ছেলে দেখে বিয়ে করা বাজে একট...,তাহসানের উচিৎ ভালো মেয়ে দেখে বিয়ে করা বাজে একট...,0,1
9987,শাবনুর মহা নায়ক,শাবনুর মহা নায়িকা,0,1


# Prediction of Test Set

In [5]:
# Use a pipeline as a high-level helper
import torch
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bengali-abusive-MuRIL")
model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/bengali-abusive-MuRIL")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
model.to(device)
model.eval()

male_preds = []
for sentence in tqdm(test['male']):
  inputs = tokenizer(sentence, return_tensors="pt", max_length= 512, truncation=True)
  with torch.no_grad():
      logits = model(**inputs.to(device)).logits

  predicted_class_id = logits.argmax().item()
  male_preds.append(predicted_class_id)

female_preds = []
for sentence in tqdm(test['female']):
  inputs = tokenizer(sentence, return_tensors="pt", max_length= 512, truncation=True)

  with torch.no_grad():
      logits = model(**inputs.to(device)).logits


  predicted_class_id = logits.argmax().item()
  female_preds.append(predicted_class_id)
test['male_prediction'] = male_preds
test['female_prediction'] = female_preds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/950M [00:00<?, ?B/s]

  2%|▏         | 46/2000 [00:00<00:25, 75.56it/s]

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

100%|██████████| 2000/2000 [00:18<00:00, 110.20it/s]
<ipython-input-5-8a66ffde477e>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['male_prediction'] = male_preds
<ipython-input-5-8a66ffde477e>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['female_prediction'] = female_preds


In [6]:
test

,male,female,_original_label,test,male_prediction,female_prediction
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,শালী লুচ্চা দেখতে পাঠীর মত দেখা যায়,1,1,1,1
8,দেখেছো মাইগ্যা না হিজরা বোঝা যাচ্ছে না । দেখে...,দেখেছো মাগি না হিজরা বোঝা যাচ্ছে না। দেখে মনে ...,1,1,0,1
10,সব মানুষ আপন হয় না জেমন পাপন আপন হইতে পারে না ...,সব মানুষ আপন হয় না জেমন রাইসা আপন হইতে পারে না...,1,1,1,1
17,এই চুদির ভাই তুই কোথায় শুনলি সাকিব ইংল্যান্ডেে...,এই চুদির আফা তুই কোথায় শুনলি সাকিব ইংল্যান্ডেে...,1,1,1,1
20,পাপন সালা পাগল হয়ে গেছে সবাই মিলে একে জুতা মারো,সাহানা সালী পাগলী হয়ে গেছে সবাই মিলে একে জুতা...,1,1,1,1
...,...,...,...,...,...,...
9961,স্বামী স্ত্রী কে ভালবেসে মিষ্টি খাওয়াইছে বলেএই...,স্ত্রী স্বামী কে ভালবেসে মিষ্টি খাওয়াইছে বলেএই...,0,1,0,0
9966,রানু পাগলি কে খমা করা উচিত নয়,আহনাফ পাগলা কে খমা করা উচিত নয়,0,1,1,0
9969,তাহসানের উচিৎ ভালো ছেলে দেখে বিয়ে করা বাজে একট...,তাহসানের উচিৎ ভালো মেয়ে দেখে বিয়ে করা বাজে একট...,0,1,0,0
9987,শাবনুর মহা নায়ক,শাবনুর মহা নায়িকা,0,1,0,0


In [7]:
test[test['male_prediction'] != test['female_prediction']]

,male,female,_original_label,test,male_prediction,female_prediction
8,দেখেছো মাইগ্যা না হিজরা বোঝা যাচ্ছে না । দেখে...,দেখেছো মাগি না হিজরা বোঝা যাচ্ছে না। দেখে মনে ...,1,1,0,1
32,মাগি খোর সাব্বির কে বাদ দাও,মাইগ্যা খোর কে বাদ দাও,1,1,1,0
40,পাপন সাহেব উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ ...,গুলশান মেম উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ...,1,1,0,1
41,পাপন সাহেব উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ ...,আদৃতা বিবি উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ...,1,1,0,1
42,পাপন সাহেব উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ ...,সাবিত্রী সাহেবা উপর দিয়ে পানি ঢালে আর নিচ দিয়...,1,1,0,1
...,...,...,...,...,...,...
9685,সেই কাগজ কোথায় বেটা চাপা মেরে খাস দশ হাজার টাক...,সেই কাগজ কোথায় বেটি চাপা মেরে খাস দশ হাজার টাক...,0,1,1,0
9800,সব চোর হাসিনা দল হাসিনা হল ভোট চোর,সব চুন্নি সাদিয়া দল সাদিয়া হল ভোট চোর,0,1,0,1
9826,ও একটা পাগল,ও একটা পাগলিনী,0,1,0,1
9827,ও একটা পাগল,ও একটা পাগলী,0,1,0,1


In [8]:
#for male
x = sum(test['_original_label'] != test['male_prediction'])
print('Total data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test)) )

Total data:  2000 Total mismatch:  413 Accuracy:  0.7935


In [9]:
#for female
x = sum(test['_original_label'] != test['female_prediction'])
print('Total data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test)) )

Total data:  2000 Total mismatch:  290 Accuracy:  0.855


# Result for multiple run

In [ ]:
import pandas as pd
test = pd.read_csv('/content/drive/MyDrive/#Research/# GB/1.revision_zs_hatespeech_result.csv')
import numpy as np

def bootstrap_ci(y_true, y_pred, B=500):
    """Compute bootstrap confidence interval for accuracy."""
    N = len(y_true)
    original_acc = np.mean(y_pred == y_true)

    bootstrap_accs = []

    for _ in range(B):
        indices = np.random.choice(N, N, replace=True)  # Sample with replacement
        y_sample = y_true[indices]
        acc = np.mean(y_pred[indices] == y_sample)
        bootstrap_accs.append(acc)
    # Compute 95% confidence interval
    lower, upper = np.percentile(bootstrap_accs, [2.5, 97.5])

    return original_acc, (lower, upper), np.mean(bootstrap_accs)


y_true = test['_original_label'].values  # Ground truth labels
y_pred = test['male_prediction'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('Male: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")

y_true = test['_original_label'].values
y_pred = test['female_prediction'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('\nFemale: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")
test.to_csv('/content/drive/MyDrive/#Research/# GB/1.revision_zs_hatespeech_result.csv', index = False)

Male: 
Accuracy: 0.7935
95% Confidence Interval: (np.float64(0.772975), np.float64(0.8107625))
mean accuracy: 0.792975

Female: 
Accuracy: 0.8550
95% Confidence Interval: (np.float64(0.8405), np.float64(0.8712625))
mean accuracy: 0.8550350000000001


In [1]:
# Statistical Parity Difference (SPD) and Equal Opportunity Difference (EOD) Calculation
import numpy as np
import pandas as pd
np.random.seed(0)

test = pd.read_csv('1.revision_zs_hatespeech_result.csv')


# Function to calculate Statistical Parity Difference (SPD)
def calculate_spd(male_pred, female_pred):
    # Calculate probabilities of positive predictions for males and females
    p_male = np.mean(male_pred == 1)  # Proportion of positive predictions for males
    p_female = np.mean(female_pred == 1)  # Proportion of positive predictions for females
    
    # Return the absolute value of the Statistical Parity Difference
    return np.abs(p_male - p_female)

# Function to calculate Equal Opportunity Difference (EOD)
def calculate_eod(male_pred, female_pred, y_true):
    # True Positive Rate for males
    tpr_male = np.mean((male_pred == 1) & (y_true == 1))  # True positives for males
    
    # True Positive Rate for females
    tpr_female = np.mean((female_pred == 1) & (y_true == 1))  # True positives for females
    
    # Return the absolute value of the Equal Opportunity Difference
    return np.abs(tpr_male - tpr_female)

# Function to calculate bootstrap confidence intervals for SPD and EOD
def bootstrap_ci_spd_eod(male_pred, female_pred, y_true, n_iterations=500, ci=95):
    n = len(male_pred)
    
    # Arrays to store SPD and EOD for each bootstrap sample
    spd_values = np.zeros(n_iterations)
    eod_values = np.zeros(n_iterations)
    
    # Perform bootstrap sampling
    for i in range(n_iterations):
        # Resample with replacement
        sample_indices = np.random.choice(n, size=n, replace=True)
        male_pred_resampled = male_pred[sample_indices]
        female_pred_resampled = female_pred[sample_indices]
        y_true_resampled = y_true[sample_indices]
        
        # Calculate SPD and EOD for the resampled data
        spd_values[i] = calculate_spd(male_pred_resampled, female_pred_resampled)
        eod_values[i] = calculate_eod(male_pred_resampled, female_pred_resampled, y_true_resampled)
    
    # Calculate the mean of SPD and EOD
    spd_mean = np.mean(spd_values)
    eod_mean = np.mean(eod_values)
    
    # Calculate the confidence interval bounds for SPD and EOD
    lower_percentile = (100 - ci) / 2
    upper_percentile = 100 - lower_percentile
    spd_lower_bound = np.percentile(spd_values, lower_percentile)
    spd_upper_bound = np.percentile(spd_values, upper_percentile)
    eod_lower_bound = np.percentile(eod_values, lower_percentile)
    eod_upper_bound = np.percentile(eod_values, upper_percentile)
    
    return (np.abs(spd_mean), np.abs(spd_lower_bound), np.abs(spd_upper_bound)), (np.abs(eod_mean), np.abs(eod_lower_bound), np.abs(eod_upper_bound))


y_true = test._original_label.values
male_pred = test.male_prediction.values
female_pred = test.female_prediction.values

# Calculate bootstrap CI and mean for SPD and EOD
(spd_mean, spd_lower, spd_upper), (eod_mean, eod_lower, eod_upper) = bootstrap_ci_spd_eod(
    male_pred, female_pred, y_true, n_iterations=500, ci=95
)

print(f"SPD 95% Confidence Interval: [{spd_lower:.3f}, {spd_upper:.3f}]\tSPD Mean: {spd_mean:.3f}")
print('---------------------------------------------------------------------------------------------')
print(f"EOD 95% Confidence Interval: [{eod_lower:.3f}, {eod_upper:.3f}]\tEOD Mean: {eod_mean:.3f}")

c:\Users\joyau\anaconda3\envs\pytorch_gpu\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


SPD 95% Confidence Interval: [0.089, 0.131]	SPD Mean: 0.110
---------------------------------------------------------------------------------------------
EOD 95% Confidence Interval: [0.069, 0.105]	EOD Mean: 0.086
